In [1]:
import tensorflow as tf

Loss function

In [2]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

# usage
# model.compile(loss=huber_fn, optimizer="nadam")

In [3]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold**2 * 0.5
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

In [4]:
class HuberLoss(tf.keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__()
    
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2 * 0.5
        return tf.where(is_small_error, squared_loss, linear_loss)
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

Activation function

In [5]:
def softplus(z):
    return tf.math.log(1.0 + tf.exp(z))

Initializer

In [6]:
def glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

Regularizer

In [7]:
def l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

Constraints

In [8]:
def positive_weights(weights):
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

Sample usage

In [9]:
layer = tf.keras.layers.Dense(1,
                              activation=softplus,
                              kernel_initializer=glorot_initializer,
                              kernel_regularizer=l1_regularizer,
                              kernel_constraint=positive_weights)

Regularizer with custom factor as class

In [10]:
class L1Regularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, factor) -> None:
        self.factor = factor

    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    
    def get_config(self):
        return {"factor": self.factor}

Estimator

In [11]:
class HuberMetric(tf.keras.metrics.Metric):
    def __init__(self, threshold, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        sample_metrics = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(sample_metrics))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))

    def result(self):
        return self.total / self.count
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

Layer (no weights)

In [ ]:
exponential_layer = tf.keras.layers.Lambda(lambda x: tf.exp(x))

Layer (with weights)

In [ ]:
class CustomDense(tf.keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = activation
    
    def build(self, input_shape):
        self.kernel = self.add_weight(name="kernel", shape=[input_shape[-1], self.units], initializer='glorot_normal')
        self.bias = self.add_weight(name="bias", shape=[self.units], initializer='zeros')
    
    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units, "activation": tf.keras.activations.serialize(self.activation)}


Custom model with residual block layer

In [ ]:
class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [tf.keras.layers.Dense(n_neurons, activation='relu', kernel_initializer='he_normal') for _ in range(n_layers)]

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z

In [ ]:
class ResidualRegressor(tf.keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hiddenl = tf.keras.layers.Dense(30, activation='relu', kernel_initializer='he_normal')
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = tf.keras.layers.Dense(output_dim)

    def call(self, inputs):
        Z = self.hiddenl(inputs)
        for _ in range(1 + 3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)